In [1]:
#import
import pandas as pd
from sklearn.preprocessing import MultiLabelBinarizer

movies = pd.read_csv('dataset-ml-25m/movies.csv', sep=',')

In [2]:
# La lista di generi che vuoi codificare
genres_list = ["Action","Adventure","Animation","Children",
"Comedy","Crime","Documentary","Drama","Fantasy","Film-Noir","Horror","IMAX",
"Musical","Mystery","Romance","Sci-Fi","Thriller","War","Western","(no genres listed)"]

# Crea un'istanza della classe MultiLabelBinarizer
mlb = MultiLabelBinarizer(classes=genres_list)

# Suddividi la colonna genres in più colonne, una per ogni genere
movies_genres = movies['genres'].str.get_dummies(sep='|')

# Seleziona solo le colonne corrispondenti ai generi presenti in genres_list
movies_genres = movies_genres[genres_list]

# Aggiungi le colonne codificate a dummy variables al dataframe originale
movies = pd.concat([movies, movies_genres], axis=1)

#drop genres column
movies = movies.drop('genres', axis=1)

In [3]:
movies.head()
print(movies)
#print movies in csv
movies.to_csv('dataset-ml-25m/movies_modify.csv')

       movieId                               title  Action  Adventure  \
0            1                    Toy Story (1995)       0          1   
1            2                      Jumanji (1995)       0          1   
2            3             Grumpier Old Men (1995)       0          0   
3            4            Waiting to Exhale (1995)       0          0   
4            5  Father of the Bride Part II (1995)       0          0   
...        ...                                 ...     ...        ...   
62418   209157                           We (2018)       0          0   
62419   209159           Window of the Soul (2001)       0          0   
62420   209163                    Bad Poems (2018)       0          0   
62421   209169                 A Girl Thing (2001)       0          0   
62422   209171      Women of Devil's Island (1962)       1          1   

       Animation  Children  Comedy  Crime  Documentary  Drama  ...  Horror  \
0              1         1       1      0    

In [4]:
#print the number of rows and columns
#print("Dataframe shape: ", movies.shape)
#print("Numero di righe:", movies.shape[0])
#print("Numero di colonne:", movies.shape[1])

#Analizza la distribuzione delle valutazioni:
#print(movies.describe())
#print the dataframe info
#print("Dataframe info: ", movies.info())

#count the null values in each column
#print("Null values: ", movies.isna().sum())

#print duplicates
#print("Duplicates: ", movies.duplicated().sum())

In [5]:
#count for each genre_list the number of movies
#print("Number of movies for each genre: ", movies[genres_list].sum())
#plot the number of movies for each genre
#movies[genres_list].sum().plot(kind='bar', figsize=(15,5), title='Number of movies for each genre')

In [6]:
# Merge tag and movies and create columns for each tag
gen_tags = pd.read_csv('dataset-ml-25m/genome-tags.csv', sep=',')
gen_scores = pd.read_csv('dataset-ml-25m/genome-scores.csv', sep=',')

#print("Null values: ", gen_tags.isna().sum())
#print("Null values: ", gen_scores.isna().sum())
#print duplicates
#print("Duplicates: ", gen_tags.duplicated().sum())
#print("Duplicates: ", gen_scores.duplicated().sum())

In [ ]:
df = movies.merge(gen_scores,on='movieId')
df = df.merge(gen_tags, on='tagId')
df =df.pivot_table(index=['movieId','title','Action','Adventure','Animation','Children','Comedy','Crime',
                'Documentary','Drama','Fantasy','Film-Noir','Horror',
                'IMAX','Musical','Mystery','Romance','Sci-Fi','Thriller','War','Western','(no genres listed)'],
                columns='tag', values='relevance', fill_value=0).reset_index().rename_axis(None,axis=1)

df.to_csv("dataset-ml-25m/movies_genome.csv",index=False)

In [ ]:
#Merge movies_genome with film rating
ratings = pd.read_csv('dataset-ml-25m/ratings.csv', sep=',')
movies = pd.read_csv('dataset-ml-25m/movies_genome.csv', sep=',')

ratings = ratings.groupby(['movieId'])['rating'].mean().reset_index()

df.to_csv("dataset-ml-25m/movies_rating.csv",index=False)

#Rounded to the nearest 0.5
def myround(x, prec=2, base=.5):
  return round(base * round(float(x)/base),prec)

ratings["rating"] = ratings["rating"].apply(lambda x:myround(x))
df = ratings.merge(movies,on='movieId')

dataset = df.drop('title', axis=1)

In [ ]:
dataset.to_csv("dataset-ml-25m/dataset.csv",index=False)